In [1]:
! pip install semantic-kernel -U 
! pip install qdrant_client -U 

In [2]:
import os
import json

In [3]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [5]:
gpt4_service = skaoai.AzureChatCompletion("GPT4Model",endpoint,api_key=api_key,api_version = "2023-12-01-preview")

embedding_gen = skaoai.AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-12-01-preview")

In [6]:
kernel.add_service(gpt4_service)


kernel.add_service(embedding_gen)

In [7]:
base_plugin = "./plugins"

In [8]:
files_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "FilePlugin")

In [9]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [10]:
kblist = []

In [11]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = await kernel.invoke(notesFunc, sk.KernelArguments(input=content))
    # result = await notesFunc(content)
    print(result) 
    json_result = json.loads(str(result))
    kblist.append(json_result)
    file.close()

{"kb": "History of machine learning", "content": "The history of machine learning and artificial intelligence is marked by significant discoveries and periods of progress and stagnation. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). Key periods include the conceptualization of 'machines that can think' by Alan Turing in 1950, the Dartmouth Summer Research Project in 1956, the 'golden years' of AI from 1956 to 1974, the 'AI Winter' from 1974 to 1980 due to limitations and criticisms, the rise of 'expert systems' in the 1980s, the 'AI Chill' from 1987 to 1993 due to over-specialization, and the resurgence of ML and AI from 1993 to 2011 due to increased data and computational power. Today, ML and AI are integral parts of our lives, necessitating careful understanding of their impacts."}
{
"kb": "Techniques of Machine Learning

In [12]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = await kernel.invoke(transcripsFunc, sk.KernelArguments(input=content))
    print(result)
    json_result = json.loads(str(result))
    for item in json_result:
        kblist.append(item)
    file.close()

[
{"kb": "Alan Turing and the Turing test", "content": "Alan Turing laid the foundation for the concept of a machine that can think. He created the Turing test in 1950, where an interrogator questions both a human and a computer and tries to determine which one is which. If the interrogator cannot tell the difference, then the computer can be considered intelligent."},
{"kb": "The Dartmouth Summer Research Project on AI", "content": "In 1956, the term artificial intelligence was coined at Dartmouth College in the U.S during an event called the summer research project on artificial intelligence. This conference was the birth of the field of research we know as AI."},
{"kb": "The golden years of AI", "content": "The years from 1956 to 1974 are known as The Golden Ears of AI. During this period, optimism ran high in the hope that AI could solve many problems. Advances were made in computation and algorithms, and prototypes of intelligent machines were built."},
{"kb": "The AI winter", "co

In [13]:
kblist

[{'kb': 'History of machine learning',
  'content': "The history of machine learning and artificial intelligence is marked by significant discoveries and periods of progress and stagnation. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). Key periods include the conceptualization of 'machines that can think' by Alan Turing in 1950, the Dartmouth Summer Research Project in 1956, the 'golden years' of AI from 1956 to 1974, the 'AI Winter' from 1974 to 1980 due to limitations and criticisms, the rise of 'expert systems' in the 1980s, the 'AI Chill' from 1987 to 1993 due to over-specialization, and the resurgence of ML and AI from 1993 to 2011 due to increased data and computational power. Today, ML and AI are integral parts of our lives, necessitating careful understanding of their impacts."},
 {'kb': 'Techniques of Machine Lear

In [14]:
base_vectordb = 'aboutMLKBDemoDemo'

In [15]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [16]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection(base_vectordb)

In [17]:
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

In [18]:
memory = SemanticTextMemory(storage=qdrant_store, embeddings_generator=embedding_gen)

In [19]:
import uuid

In [20]:
uuid.uuid4()

UUID('19852eeb-3b9c-4c70-9591-6f6d7af9f823')

In [21]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await memory.save_information(base_vectordb, id=id, text=content)

In [22]:
ask = "can you tell me what is different ML and AI"

memories = await memory.search(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [23]:
result = ''
for memory in memories:
    print(f"Top Result: {memory.text} with score {memory.relevance}")
    result = memory.text

Top Result: The difference between AI and ML - AI is about machines accomplishing tasks requiring human-level intelligence. Machine learning, a subset of AI, uses algorithms to make decisions from data. Deep learning, a subset of machine learning, uses neural networks. with score 0.89723134


In [24]:
answer_plugin = kernel.import_plugin_from_prompt_directory(base_plugin , "AnswerPlugin")

In [25]:
sumFunc = answer_plugin["Summary"]

In [26]:

summary_result = await kernel.invoke(sumFunc, sk.KernelArguments(input=result))

In [29]:
str(summary_result)

'AI, or Artificial Intelligence, refers to machines performing tasks that usually require human intelligence. Machine Learning, a part of AI, involves using algorithms to make decisions based on data. Deep Learning, a subset of Machine Learning, utilizes neural networks. In essence, all three are interconnected, with AI being the broadest concept, Machine Learning being a method to achieve AI, and Deep Learning being a specialized type of Machine Learning.'